

---


# **Research**


---



**Title:** Examining Input Noise Injection During Dropout in AlexNet for Robust CIFAR Dataset Image Recognition

**Objective:** This research aims to investigate the impact of input noise injection on dropped neurons during the training phase of AlexNet, which is a pioneering convolutional neural network (CNN) for image classification tasks. We will use the CIFAR-10 and CIFAR-100 datasets to determine how input noise affects AlexNet's resilience to adversarial attacks and its ability to accurately classify images




---
### **Methodology:**



---



**Model Framework:** We will be working with AlexNet, a cornerstone in the development of CNNs for categorizing images.

**Datasets for Testing:** The CIFAR-10 and CIFAR-100 datasets, encompassing a broad spectrum of image classes, will serve as our test bed to rigorously evaluate the model's robustness and consistency.

**Noise Injection Strategy:** Our approach involves injecting stochastic noise directly to the input data corresponding to neurons that are dropped during the training process. This will help us assess how the network copes with the added uncertainty and whether this can enhance its robustness.

**Training and Evaluation:** We will meticulously train AlexNet from the ground up, meticulously tuning the noise injection parameters and observing how these adjustments influence the network's learning efficacy.

**Performance Metrics:** The success of our noise injection will be measured by contrasting the network's performance on standard testing images against its performance when challenged with adversarially modified images.

**Update on the testing strategy:** We First determined the standard deviation of the dataset, which was 0.1595, and used this value as the base rate for applying noise. Noise levels were introduced at 5% (0.007975), 10% (0.01595), 30% (0.04785), and 50% (0.07975) relative to this base rate. For dropout probabilities, we applied the same series of noise levels across three different settings: at a 20% dropout probability, we employed noise rates of 5%, 10%, 30%, 50%, and 100% of the base rate; this pattern was consistently used for dropout probabilities of 25% and 30% as well.




---


### **Anticipated Challenges:**


---



**Stabilizing the Model:** Injecting noise at the input level may introduce new complexities in the network’s learning process, necessitating careful recalibration of the training hyperparameters.

**Noise Optimization:** We aim to pinpoint the precise noise level that strikes a balance between fortifying robustness and maintaining high accuracy on standard image sets.

**Computational Demands:** The extensive computational resources required for training and evaluating numerous variations of the model under different noise conditions represent a significant logistical challenge.


### **Expected Outcomes:**


**Robustness Advancements:** We anticipate our method will significantly boost AlexNet's defenses against deceptive inputs, leading to more reliable real-world application performance.

**Insightful Discoveries:** This exploration is poised to shed light on the dynamics of noise in neural network training and its potential to create more resilient network architectures.

### **Contribution to the Field:**


With this project, we aim to enrich the neural network community with insights and methodologies that enhance the security of CNNs like AlexNet. Our findings could pave the way for new, more robust training techniques that harden neural networks against sophisticated adversarial tactics, especially in fields where security and reliability are paramount.

## **Setting Up the Environment**


In [ ]:
!pip install torch torchvision torchaudio #installing pytorch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

###**Standard Testing**

In [ ]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision.datasets import CIFAR10, MNIST
from torchvision.transforms.v2 import ToTensor
from torchvision import datasets, transforms

In [ ]:
# CIFAR-10 transform - three channels, normalize with 3 means and 3 SDs
cifar_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# CIFAR10 data
train_dataset = CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=cifar_transform
)

val_dataset = CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=cifar_transform
)

100%|██████████| 170498071/170498071 [00:02<00:00, 62801218.54it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# dataloaders
batch_size = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
#Compute Standard Deviation of the Dataset

def compute_dataset_std(train_loader):
    # We square sum the standard deviations of each batch to compute the total variance, then take the square root.
    var_sum = 0
    count = 0
    for data, _ in train_loader:
        var_sum += data.var([0, 2, 3], unbiased=False).sum()
        count += data.size(0)
    total_variance = var_sum / count
    dataset_std = torch.sqrt(total_variance)
    return dataset_std.mean()

dataset_std = compute_dataset_std(train_loader)
print("Computed standard deviation of the dataset:", dataset_std)


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Computed standard deviation of the dataset: tensor(0.1529)


In [ ]:
# Loss fuction and optimizer
def get_crit_and_opt(net):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    return criterion, optimizer

CIFAR10net

In [ ]:
class CIFAR10Net(nn.Module):
    def __init__(self, dropout_rate=0.25):
        super(CIFAR10Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)
        dummy_input = torch.autograd.Variable(torch.zeros(1, 3, 32, 32))
        dummy_output = self.pool(self.conv4(self.pool(self.conv3(self.pool(self.conv2(self.pool(self.conv1(dummy_input))))))))
        self.final_feature_map_size = dummy_output.size(-1) * dummy_output.size(-2) * dummy_output.size(-3)
        self.fc1 = nn.Linear(self.final_feature_map_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.relu(self.pool(self.conv3(x)))
        x = self.dropout1(x)
        x = F.relu(self.pool(self.conv4(x)))
        x = self.dropout2(x)
        x = x.view(-1, self.final_feature_map_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


In [ ]:
class AverageMeter(object):

    """Computes and stores an average and current value."""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def error_rate(output, target, topk=(1,)):

    """Computes the top-k error rate for the specified values of k."""

    maxk = max(topk) # largest k we'll need to work with
    batch_size = target.size(0) # determine batch size

    # get maxk best predictions for each item in the batch, both values and indices
    _, pred = output.topk(maxk, 1, True, True)

    # reshape predictions and targets and compare them element-wise
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk: # for each top-k accuracy we want

        # num correct
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        # num incorrect
        wrong_k = batch_size - correct_k
        # as a percentage
        res.append(wrong_k.mul_(100.0 / batch_size))

    return res

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# training function - 1 epoch
def train(
    train_loader,
    model,
    criterion,
    optimizer,
    epoch,
    epochs,
    print_freq = 100,
    verbose = True
):

    # track average and worst losses
    losses = AverageMeter()

    # set training mode
    model.train()

    # iterate over data - automatically shuffled
    for i, (images, labels) in enumerate(train_loader):

        # put batch of image tensors on GPU
        images = images.to(device)
        # put batch of label tensors on GPU
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # model output
        outputs = model(images)

        # loss computation
        loss = criterion(outputs, labels)

        # back propagation
        loss.backward()

        # update model parameters
        optimizer.step()

        # update meter with the value of the loss once for each item in the batch
        losses.update(loss.item(), images.size(0))

        # logging during epoch
        if i % print_freq == 0 and verbose == True:
            print(
                f'Epoch: [{epoch+1}/{epochs}][{i:4}/{len(train_loader)}]\t'
                f'Loss: {losses.val:.4f} ({losses.avg:.4f} on avg)'
            )

    # log again at end of epoch
    print(f'\n* Epoch: [{epoch+1}/{epochs}]\tTrain loss: {losses.avg:.3f}\n')

    return losses.avg

In [ ]:
# val function
def validate(
    val_loader,
    model,
    criterion,
    epoch,
    epochs,
    print_freq = 100,
    verbose = True
):

    # track average and worst losses and batch-wise top-1 and top-5 accuracies
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # set evaluation mode
    model.eval()

    # iterate over data - automatically shuffled
    for i, (images, labels) in enumerate(val_loader):

        # put batch of image tensors on GPU
        images = images.to(device)
        # put batch of label tensors on GPU
        labels = labels.to(device)

        # model output
        output = model(images)

        # loss computation
        loss = criterion(output, labels)

        # top-1 and top-5 accuracy on this batch
        err1, err5, = error_rate(output.data, labels, topk=(1, 5))

        # update meters with the value of the loss once for each item in the batch
        losses.update(loss.item(), images.size(0))
        # update meters with top-1 and top-5 accuracy on this batch once for each item in the batch
        top1.update(err1.item(), images.size(0))
        top5.update(err5.item(), images.size(0))

        # logging during epoch
        if i % print_freq == 0 and verbose == True:
            print(
                f'Test (on val set): [{epoch+1}/{epochs}][{i:4}/{len(val_loader)}]\t'
                f'Loss: {losses.val:.4f} ({losses.avg:.4f} on avg)\t'
                f'Top-1 err: {top1.val:.4f} ({top1.avg:.4f} on avg)\t'
                f'Top-5 err: {top5.val:.4f} ({top5.avg:.4f} on avg)'
            )

    # logging for end of epoch
    print(
        f'\n* Epoch: [{epoch+1}/{epochs}]\t'
        f'Test loss: {losses.avg:.3f}\t'
        f'Top-1 err: {top1.avg:.3f}\t'
        f'Top-5 err: {top5.avg:.3f}\n'
    )

    # avergae top-1 and top-5 accuracies batch-wise, and average loss batch-wise
    return top1.avg, top5.avg, losses.avg

In [ ]:
# best error rates so far
best_err1 = 100
best_err5 = 100

In [ ]:
# Run the main function.
if __name__ == '__main__':

    # select a model to train here (CIFAR10Net or MNISTNet)
    model = CIFAR10Net()

    # move to GPU
    model.to(device)

    # select number of epochs
    epochs = 10

    # get criterion and optimizer
    criterion, optimizer = get_crit_and_opt(model)

    # epoch loop
    for epoch in range(0, epochs):

        # train for one epoch
        train_loss = train(
          train_loader,
          model,
          criterion,
          optimizer,
          epoch,
          epochs
        )

        # evaluate on validation set
        err1, err5, val_loss = validate(
          val_loader,
          model,
          criterion,
          epoch,
          epochs
        )

        # remember best prec@1 and save checkpoint
        is_best = err1 <= best_err1
        best_err1 = min(err1, best_err1)
        if is_best:
            best_err5 = err5

        print('Current best error rate (top-1 and top-5 error):', best_err1, best_err5, '\n')
    print('Best error rate (top-1 and top-5 error):', best_err1, best_err5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: [1/10][   0/1563]	Loss: 2.3070 (2.3070 on avg)
Epoch: [1/10][ 100/1563]	Loss: 2.3023 (2.3033 on avg)
Epoch: [1/10][ 200/1563]	Loss: 2.3026 (2.3031 on avg)
Epoch: [1/10][ 300/1563]	Loss: 2.3011 (2.3028 on avg)
Epoch: [1/10][ 400/1563]	Loss: 2.3039 (2.3026 on avg)
Epoch: [1/10][ 500/1563]	Loss: 2.2998 (2.3025 on avg)
Epoch: [1/10][ 600/1563]	Loss: 2.3012 (2.3022 on avg)
Epoch: [1/10][ 700/1563]	Loss: 2.3023 (2.3021 on avg)
Epoch: [1/10][ 800/1563]	Loss: 2.3032 (2.3019 on avg)
Epoch: [1/10][ 900/1563]	Loss: 2.2975 (2.3017 on avg)
Epoch: [1/10][1000/1563]	Loss: 2.3008 (2.3014 on avg)
Epoch: [1/10][1100/1563]	Loss: 2.3036 (2.3011 on avg)
Epoch: [1/10][1200/1563]	Loss: 2.2981 (2.3007 on avg)
Epoch: [1/10][1300/1563]	Loss: 2.2958 (2.3001 on avg)
Epoch: [1/10][1400/1563]	Loss: 2.2740 (2.2992 on avg)
Epoch: [1/10][1500/1563]	Loss: 2.2727 (2.2981 on avg)


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



* Epoch: [1/10]	Train loss: 2.297

Test (on val set): [1/10][   0/313]	Loss: 2.2467 (2.2467 on avg)	Top-1 err: 90.6250 (90.6250 on avg)	Top-5 err: 31.2500 (31.2500 on avg)
Test (on val set): [1/10][ 100/313]	Loss: 2.2311 (2.2575 on avg)	Top-1 err: 93.7500 (84.9010 on avg)	Top-5 err: 21.8750 (36.8812 on avg)
Test (on val set): [1/10][ 200/313]	Loss: 2.2832 (2.2572 on avg)	Top-1 err: 93.7500 (84.9658 on avg)	Top-5 err: 50.0000 (36.3806 on avg)
Test (on val set): [1/10][ 300/313]	Loss: 2.2676 (2.2573 on avg)	Top-1 err: 87.5000 (84.7799 on avg)	Top-5 err: 34.3750 (36.5864 on avg)

* Epoch: [1/10]	Test loss: 2.257	Top-1 err: 84.780	Top-5 err: 36.560

Current best error rate (top-1 and top-5 error): 25.52 1.93 

Epoch: [2/10][   0/1563]	Loss: 2.2773 (2.2773 on avg)
Epoch: [2/10][ 100/1563]	Loss: 2.2673 (2.2408 on avg)
Epoch: [2/10][ 200/1563]	Loss: 2.1685 (2.2040 on avg)
Epoch: [2/10][ 300/1563]	Loss: 1.8824 (2.1580 on avg)
Epoch: [2/10][ 400/1563]	Loss: 2.1473 (2.1299 on avg)
Epoch: [2/10]

In [ ]:
# Create a classification report for one model [10 pts]
from sklearn.metrics import classification_report
import numpy as np


y_true = []
y_pred = []

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation for efficiency
with torch.no_grad():
    for images, labels in val_loader:
        # Move tensors to the appropriate device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass to get the model's predictions
        outputs = model(images)

        # Convert outputs to predicted class by taking the index with the maximum score in each output row
        _, predicted = torch.max(outputs, 1)

        # Append true and predicted labels to lists
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Convert lists to arrays for compatibility with classification_report
y_true = np.array(y_true)
y_pred = np.array(y_pred)

#___________________________________________________________________________________________________________________________

# Assuming you have 10 classes for CIFAR10, and their names as follows:
target_names = ['airplane', 'bird', 'vegetable', 'dog', 'cat',
                'car', 'fruit', 'train', 'rabbit', 'baby']

# Generate the classification report
report = classification_report(y_true, y_pred, target_names=target_names)

print(report)

              precision    recall  f1-score   support

    airplane       0.74      0.69      0.72      1000
        bird       0.90      0.75      0.82      1000
   vegetable       0.59      0.54      0.56      1000
         dog       0.50      0.26      0.34      1000
         cat       0.71      0.52      0.60      1000
         car       0.44      0.78      0.56      1000
       fruit       0.63      0.88      0.73      1000
       train       0.69      0.75      0.72      1000
      rabbit       0.91      0.70      0.79      1000
        baby       0.75      0.79      0.77      1000

    accuracy                           0.67     10000
   macro avg       0.69      0.67      0.66     10000
weighted avg       0.69      0.67      0.66     10000



6. Evaluation and Analysis
Evaluate the Model: Test the model on both normal and adversarially modified images.
Plot Results: Use Matplotlib to visualize the training and validation results.

7. Documentation and Saving Work
Document Your Work: Utilize Markdown cells in Google Colab to describe each part of your process.
Save Your Model and Data: Ensure to save your trained model and any important data for later use.

### **Error Injection Testing**




In models that use dropout, different neurons are randomly dropped in each forward pass during training. This means that with each batch during an epoch, and indeed across epochs, different sets of neurons might be turned off (i.e., their activations are set to zero or replaced with noise in the case of your NoisyDropout). This randomness is key to how dropout enhances the generalization capabilities of neural networks by preventing them from being overly reliant on any particular neuron or feature detector, thereby reducing overfitting.

How Dropout Works with Noise
When using the NoisyDropout class that you've implemented, here's what happens during each training iteration (not just each epoch):

Binary Mask Generation: A random binary mask is generated where the probability of a neuron being zeroed out (dropped) is determined by the dropout probability p. This mask is different for each forward pass.

Noise Application: For the neurons that are dropped (where the mask is True), random Gaussian noise is added. This noise is generated anew each time, so its values are different in each forward pass.

Forward Pass Application: The forward method of NoisyDropout applies this mask and noise to the input tensor x, modifying the activations accordingly.

Ensuring Neurons Receive New Noise Each Epoch
The process you've set up with NoisyDropout inherently ensures that each time an epoch progresses and for each batch within that epoch, the dropped neurons receive new noise. This is due to the stochastic nature of both the dropout mask and the noise generation occurring within the forward method of the dropout layer. Each call to the forward method generates a new dropout mask and new noise based on the current state of the random number generator.





In [ ]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision.datasets import CIFAR10, MNIST
from torchvision.transforms.v2 import ToTensor
from torchvision import datasets, transforms

In [ ]:
# CIFAR-10 transform - three channels, normalize with 3 means and 3 SDs
cifar_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# CIFAR10 data
train_dataset = CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=cifar_transform
)

val_dataset = CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=cifar_transform
)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# dataloaders
batch_size = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)

In [ ]:
#Compute Standard Deviation of the Dataset

def compute_dataset_std(train_loader):
    # We square sum the standard deviations of each batch to compute the total variance, then take the square root.
    var_sum = 0
    count = 0
    for data, _ in train_loader:
        var_sum += data.var([0, 2, 3], unbiased=False).sum()
        count += data.size(0)
    total_variance = var_sum / count
    dataset_std = torch.sqrt(total_variance)
    return dataset_std.mean()

dataset_std = compute_dataset_std(train_loader)
print("Computed standard deviation of the dataset:", dataset_std)


Computed standard deviation of the dataset: tensor(0.1529)


In [ ]:
# Loss fuction and optimizer
def get_crit_and_opt(net):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    return criterion, optimizer

There are two primary parameters that control the behavior of the dropout and the noise:

p (Dropout Probability): This parameter specifies the probability with which each neuron is dropped (i.e., turned off). Setting this parameter to a higher value increases the number of neurons that are dropped during each forward pass.

std_dev (Standard Deviation of Noise): This parameter controls the amount of noise added to the neurons that are dropped. A higher standard deviation results in greater noise being added, which increases the variability and randomness introduced into the model during training.

In [ ]:
# Noisy Dropout Class Definition
class NoisyDropout(nn.Module):
    def __init__(self, p=0.25, std_dev=0.1529):
        super(NoisyDropout, self).__init__()
        self.p = p  # Dropout probability
        self.std_dev = std_dev  # Standard deviation of the Gaussian noise

    def forward(self, x):
        if not self.training:
            return x # During evaluation, dropout is bypassed entirely.
        else:
            # Create a dropout mask with the same shape as the input
            # Generate a random dropout mask that has 'True' with probability 'p' (dropout rate)
            dropout_mask = torch.rand_like(x) < self.p

            # Generate Gaussian noise with the same shape as the input
            # Generate Gaussian noise with the same shape as the input tensor.
            noise = torch.randn_like(x) * self.std_dev

            # Apply noise where the dropout mask is True, otherwise keep the neuron's original value
            # Apply noise where the dropout mask is 'True'; keep original x values where 'False'.
            return torch.where(dropout_mask, noise, x)


Adjusting the Noise Rate or Ratio
To adjust the rate at which noise affects the model, you can modify the p and std_dev parameters when you instantiate the NoisyDropout layer within your network. Here's how you can do this in a model:

In [ ]:
class CIFAR10Net(nn.Module):
    def __init__(self, dropout_p=0.25, noise_std=0.1529):
        super(CIFAR10Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # NoisyDropout layers
        #self.noisy_dropout1 = NoisyDropout(p=dropout_p, std_dev=noise_std)
        #self.noisy_dropout2 = NoisyDropout(p=dropout_p, std_dev=noise_std)
        self.noisy_dropout3 = NoisyDropout(p=dropout_p, std_dev=noise_std)
        self.noisy_dropout4 = NoisyDropout(p=dropout_p, std_dev=noise_std)

        # Dummy input to determine size of the output from final conv layer
        dummy_input = torch.autograd.Variable(torch.zeros(1, 3, 32, 32))
        dummy_output = self.pool(self.conv4(self.pool(self.conv3(self.pool(self.conv2(self.pool(self.conv1(dummy_input))))))))
        self.final_feature_map_size = dummy_output.size(-1) * dummy_output.size(-2) * dummy_output.size(-3)

        self.fc1 = nn.Linear(self.final_feature_map_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        #x = self.noisy_dropout1(x)  # Apply NoisyDropout after first pooling
        x = F.relu(self.pool(self.conv2(x)))
        #x = self.noisy_dropout2(x)  # Apply NoisyDropout after second pooling
        x = F.relu(self.pool(self.conv3(x)))
        x = self.noisy_dropout3(x)  # Apply NoisyDropout after third pooling
        x = F.relu(self.pool(self.conv4(x)))
        x = self.noisy_dropout4(x)  # Apply NoisyDropout after fourth pooling

        x = x.view(-1, self.final_feature_map_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


In [ ]:
class AverageMeter(object):

    """Computes and stores an average and current value."""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def error_rate(output, target, topk=(1,)):

    """Computes the top-k error rate for the specified values of k."""

    maxk = max(topk) # largest k we'll need to work with
    batch_size = target.size(0) # determine batch size

    # get maxk best predictions for each item in the batch, both values and indices
    _, pred = output.topk(maxk, 1, True, True)

    # reshape predictions and targets and compare them element-wise
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk: # for each top-k accuracy we want

        # num correct
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        # num incorrect
        wrong_k = batch_size - correct_k
        # as a percentage
        res.append(wrong_k.mul_(100.0 / batch_size))

    return res

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# training function - 1 epoch
def train(
    train_loader,
    model,
    criterion,
    optimizer,
    epoch,
    epochs,
    print_freq = 100,
    verbose = True
):

    # track average and worst losses
    losses = AverageMeter()

    # set training mode
    model.train()

    # iterate over data - automatically shuffled
    for i, (images, labels) in enumerate(train_loader):

        # put batch of image tensors on GPU
        images = images.to(device)
        # put batch of label tensors on GPU
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # model output
        outputs = model(images)

        # loss computation
        loss = criterion(outputs, labels)

        # back propagation
        loss.backward()

        # update model parameters
        optimizer.step()

        # update meter with the value of the loss once for each item in the batch
        losses.update(loss.item(), images.size(0))

        # logging during epoch
        if i % print_freq == 0 and verbose == True:
            print(
                f'Epoch: [{epoch+1}/{epochs}][{i:4}/{len(train_loader)}]\t'
                f'Loss: {losses.val:.4f} ({losses.avg:.4f} on avg)'
            )

    # log again at end of epoch
    print(f'\n* Epoch: [{epoch+1}/{epochs}]\tTrain loss: {losses.avg:.3f}\n')

    return losses.avg

In [ ]:
# val function
def validate(
    val_loader,
    model,
    criterion,
    epoch,
    epochs,
    print_freq = 100,
    verbose = True
):

    # track average and worst losses and batch-wise top-1 and top-5 accuracies
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # set evaluation mode
    model.eval()

    # iterate over data - automatically shuffled
    for i, (images, labels) in enumerate(val_loader):

        # put batch of image tensors on GPU
        images = images.to(device)
        # put batch of label tensors on GPU
        labels = labels.to(device)

        # model output
        output = model(images)

        # loss computation
        loss = criterion(output, labels)

        # top-1 and top-5 accuracy on this batch
        err1, err5, = error_rate(output.data, labels, topk=(1, 5))

        # update meters with the value of the loss once for each item in the batch
        losses.update(loss.item(), images.size(0))
        # update meters with top-1 and top-5 accuracy on this batch once for each item in the batch
        top1.update(err1.item(), images.size(0))
        top5.update(err5.item(), images.size(0))

        # logging during epoch
        if i % print_freq == 0 and verbose == True:
            print(
                f'Test (on val set): [{epoch+1}/{epochs}][{i:4}/{len(val_loader)}]\t'
                f'Loss: {losses.val:.4f} ({losses.avg:.4f} on avg)\t'
                f'Top-1 err: {top1.val:.4f} ({top1.avg:.4f} on avg)\t'
                f'Top-5 err: {top5.val:.4f} ({top5.avg:.4f} on avg)'
            )

    # logging for end of epoch
    print(
        f'\n* Epoch: [{epoch+1}/{epochs}]\t'
        f'Test loss: {losses.avg:.3f}\t'
        f'Top-1 err: {top1.avg:.3f}\t'
        f'Top-5 err: {top5.avg:.3f}\n'
    )

    # avergae top-1 and top-5 accuracies batch-wise, and average loss batch-wise
    return top1.avg, top5.avg, losses.avg

In [ ]:
# best error rates so far
best_err1 = 100
best_err5 = 100

In [ ]:
# Run the main function.
if __name__ == '__main__':

    # select a model to train here (CIFAR10Net or MNISTNet)
    model = CIFAR10Net()

    # move to GPU
    model.to(device)

    # select number of epochs
    epochs = 10

    # get criterion and optimizer
    criterion, optimizer = get_crit_and_opt(model)

    # epoch loop
    for epoch in range(0, epochs):

        # train for one epoch
        train_loss = train(
          train_loader,
          model,
          criterion,
          optimizer,
          epoch,
          epochs
        )

        # evaluate on validation set
        err1, err5, val_loss = validate(
          val_loader,
          model,
          criterion,
          epoch,
          epochs
        )

        # remember best prec@1 and save checkpoint
        is_best = err1 <= best_err1
        best_err1 = min(err1, best_err1)
        if is_best:
            best_err5 = err5

        print('Current best error rate (top-1 and top-5 error):', best_err1, best_err5, '\n')
    print('Best error rate (top-1 and top-5 error):', best_err1, best_err5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: [1/10][   0/1563]	Loss: 2.3011 (2.3011 on avg)
Epoch: [1/10][ 100/1563]	Loss: 2.3015 (2.3030 on avg)
Epoch: [1/10][ 200/1563]	Loss: 2.3021 (2.3029 on avg)
Epoch: [1/10][ 300/1563]	Loss: 2.3031 (2.3026 on avg)
Epoch: [1/10][ 400/1563]	Loss: 2.3003 (2.3024 on avg)
Epoch: [1/10][ 500/1563]	Loss: 2.3023 (2.3024 on avg)
Epoch: [1/10][ 600/1563]	Loss: 2.3080 (2.3024 on avg)
Epoch: [1/10][ 700/1563]	Loss: 2.3024 (2.3023 on avg)
Epoch: [1/10][ 800/1563]	Loss: 2.3002 (2.3022 on avg)
Epoch: [1/10][ 900/1563]	Loss: 2.2999 (2.3021 on avg)
Epoch: [1/10][1000/1563]	Loss: 2.3028 (2.3020 on avg)
Epoch: [1/10][1100/1563]	Loss: 2.2984 (2.3018 on avg)
Epoch: [1/10][1200/1563]	Loss: 2.2996 (2.3016 on avg)
Epoch: [1/10][1300/1563]	Loss: 2.3012 (2.3014 on avg)
Epoch: [1/10][1400/1563]	Loss: 2.2930 (2.3011 on avg)
Epoch: [1/10][1500/1563]	Loss: 2.3024 (2.3007 on avg)


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



* Epoch: [1/10]	Train loss: 2.300

Test (on val set): [1/10][   0/313]	Loss: 2.2728 (2.2728 on avg)	Top-1 err: 84.3750 (84.3750 on avg)	Top-5 err: 46.8750 (46.8750 on avg)
Test (on val set): [1/10][ 100/313]	Loss: 2.2738 (2.2876 on avg)	Top-1 err: 71.8750 (85.1176 on avg)	Top-5 err: 21.8750 (45.5136 on avg)
Test (on val set): [1/10][ 200/313]	Loss: 2.2820 (2.2872 on avg)	Top-1 err: 90.6250 (85.2767 on avg)	Top-5 err: 37.5000 (45.3980 on avg)
Test (on val set): [1/10][ 300/313]	Loss: 2.2871 (2.2871 on avg)	Top-1 err: 93.7500 (85.1640 on avg)	Top-5 err: 56.2500 (45.2554 on avg)

* Epoch: [1/10]	Test loss: 2.287	Top-1 err: 85.060	Top-5 err: 45.220

Current best error rate (top-1 and top-5 error): 32.97 3.07 

Epoch: [2/10][   0/1563]	Loss: 2.3012 (2.3012 on avg)
Epoch: [2/10][ 100/1563]	Loss: 2.2902 (2.2891 on avg)
Epoch: [2/10][ 200/1563]	Loss: 2.2798 (2.2849 on avg)
Epoch: [2/10][ 300/1563]	Loss: 2.2479 (2.2787 on avg)
Epoch: [2/10][ 400/1563]	Loss: 2.3239 (2.2712 on avg)
Epoch: [2/10]

In [ ]:
# Possible values for dropout probability and noise standard deviation
dropout_probs = [0.25]
noise_stds = [0.1529]

# Function to run an experiment
def run_experiment(dropout_p, noise_std, epochs=10):
    # Initialize the model with specified dropout and noise parameters
    model = CIFAR10Net(dropout_p=dropout_p, noise_std=noise_std)
    model.to(device)

    # Criterion and optimizer
    criterion, optimizer = get_crit_and_opt(model)

    # Training and validation
    for epoch in range(epochs):
        train_loss = train(train_loader, model, criterion, optimizer, epoch, epochs)
        err1, err5, val_loss = validate(val_loader, model, criterion, epoch, epochs)

        # Logging the results
        print(f"Dropout: {dropout_p}, Noise: {noise_std}, Epoch: {epoch+1}, Train Loss: {train_loss}, Val Loss: {val_loss}, Top-1 Error: {err1}, Top-5 Error: {err5}")

# Running all experiments
for dropout_p in dropout_probs:
    for noise_std in noise_stds:
        run_experiment(dropout_p, noise_std)


Epoch: [1/10][   0/1563]	Loss: 2.3077 (2.3077 on avg)
Epoch: [1/10][ 100/1563]	Loss: 2.3020 (2.3029 on avg)
Epoch: [1/10][ 200/1563]	Loss: 2.3101 (2.3026 on avg)
Epoch: [1/10][ 300/1563]	Loss: 2.2959 (2.3027 on avg)
Epoch: [1/10][ 400/1563]	Loss: 2.3103 (2.3027 on avg)
Epoch: [1/10][ 500/1563]	Loss: 2.3017 (2.3027 on avg)
Epoch: [1/10][ 600/1563]	Loss: 2.3058 (2.3027 on avg)
Epoch: [1/10][ 700/1563]	Loss: 2.3009 (2.3027 on avg)
Epoch: [1/10][ 800/1563]	Loss: 2.3003 (2.3027 on avg)
Epoch: [1/10][ 900/1563]	Loss: 2.3035 (2.3027 on avg)
Epoch: [1/10][1000/1563]	Loss: 2.3025 (2.3026 on avg)
Epoch: [1/10][1100/1563]	Loss: 2.3012 (2.3026 on avg)
Epoch: [1/10][1200/1563]	Loss: 2.3015 (2.3026 on avg)
Epoch: [1/10][1300/1563]	Loss: 2.3016 (2.3025 on avg)
Epoch: [1/10][1400/1563]	Loss: 2.3022 (2.3025 on avg)
Epoch: [1/10][1500/1563]	Loss: 2.3009 (2.3024 on avg)

* Epoch: [1/10]	Train loss: 2.302

Test (on val set): [1/10][   0/313]	Loss: 2.3018 (2.3018 on avg)	Top-1 err: 93.7500 (93.7500 on avg)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

def generate_classification_report(model, val_loader, device):
    model.eval()  # Set the model to evaluation mode
    y_true = []
    y_pred = []

    with torch.no_grad():  # Disable gradient calculation for efficiency
        for images, labels in val_loader:
            images = images.to(device)  # Move tensors to the appropriate device
            labels = labels.to(device)

            outputs = model(images)  # Forward pass to get the model's predictions
            _, predicted = torch.max(outputs, 1)  # Convert outputs to predicted class

            y_true.extend(labels.cpu().numpy())  # Append true labels to list
            y_pred.extend(predicted.cpu().numpy())  # Append predicted labels to list

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Assuming you have 10 classes for CIFAR10, and their names as follows:
    target_names = ['airplane', 'bird', 'vegetable', 'dog', 'cat', 'car', 'fruit', 'train', 'rabbit', 'baby']

    # Generate the classification report
    report = classification_report(y_true, y_pred, target_names=target_names)
    return report

# Example usage within an experiment function
def run_experiment(dropout_p, noise_std, epochs=10):
    model = CIFAR10Net(dropout_p=dropout_p, noise_std=noise_std)
    model.to(device)

    criterion, optimizer = get_crit_and_opt(model)

    for epoch in range(epochs):
        train_loss = train(train_loader, model, criterion, optimizer, epoch, epochs)
        err1, err5, val_loss = validate(val_loader, model, criterion, epoch, epochs)

    # Generate and print classification report at the end of training
    report = generate_classification_report(model, val_loader, device)
    print(report)


In [ ]:
# Create a classification report for one model
from sklearn.metrics import classification_report
import numpy as np


y_true = []
y_pred = []

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation for efficiency
with torch.no_grad():
    for images, labels in val_loader:
        # Move tensors to the appropriate device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass to get the model's predictions
        outputs = model(images)

        # Convert outputs to predicted class by taking the index with the maximum score in each output row
        _, predicted = torch.max(outputs, 1)

        # Append true and predicted labels to lists
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Convert lists to arrays for compatibility with classification_report
y_true = np.array(y_true)
y_pred = np.array(y_pred)

#___________________________________________________________________________________________________________________________

# Assuming you have 10 classes for CIFAR10, and their names as follows:
target_names = ['airplane', 'bird', 'vegetable', 'dog', 'cat',
                'car', 'fruit', 'train', 'rabbit', 'baby']

# Generate the classification report
report = classification_report(y_true, y_pred, target_names=target_names)

print(report)

              precision    recall  f1-score   support

    airplane       0.39      0.01      0.02      1000
        bird       0.00      0.00      0.00      1000
   vegetable       0.00      0.00      0.00      1000
         dog       0.00      0.00      0.00      1000
         cat       0.00      0.00      0.00      1000
         car       0.00      0.00      0.00      1000
       fruit       0.00      0.00      0.00      1000
       train       0.00      0.00      0.00      1000
      rabbit       0.10      1.00      0.18      1000
        baby       0.00      0.00      0.00      1000

    accuracy                           0.10     10000
   macro avg       0.05      0.10      0.02     10000
weighted avg       0.05      0.10      0.02     10000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Potential Enhancements:
Dynamic Learning Rate Adjustment:

Implement learning rate schedulers to adjust the learning rate based on training epochs or loss plateaus. This can lead to better training outcomes by adapting the learning rate during training.
Augmentation and Regularization:

Consider increasing dataset diversity through more advanced data augmentation techniques, which can be particularly effective for preventing overfitting in image recognition tasks.
Additional regularization techniques, besides dropout (like L2 regularization), could also be beneficial.
Error Handling and User Feedback:

Include error handling for file operations (e.g., loading a checkpoint that does not exist) and provide clear messages to the user about what the script is doing or if something goes wrong.
Testing and Validation:

Implement separate test dataset evaluations if possible, to ensure the model's performance is validated on completely unseen data, which provides a more unbiased evaluation of the model.